In [1]:
!pip install gplearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gplearn.genetic import SymbolicRegressor
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sympy import *
from sklearn.utils.random import check_random_state
import graphviz
import time

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://github.com/Sachinjp14/Thesis/blob/3f5b58c56b972c9cbc3eee1864452e9be9b723a4/equation%204.csv?raw=true")

df.head()


,X1,X2,X3,X4,Y
0,4.725219,1.735492,4.956655,1.984024,6.442611
1,2.948350,3.917977,1.122311,2.076565,3.268741
2,2.056012,4.279370,3.592577,2.878457,3.040191
3,4.479831,4.364756,1.614712,3.498773,7.153415
4,3.310144,1.695486,1.332648,2.947337,3.784738


In [4]:
df = df.dropna()

In [5]:
df

,X1,X2,X3,X4,Y
0,4.725219,1.735492,4.956655,1.984024,6.442611
1,2.948350,3.917977,1.122311,2.076565,3.268741
2,2.056012,4.279370,3.592577,2.878457,3.040191
3,4.479831,4.364756,1.614712,3.498773,7.153415
4,3.310144,1.695486,1.332648,2.947337,3.784738
...,...,...,...,...,...
99995,3.596992,4.233839,1.935430,3.787543,6.270531
99996,2.140851,2.861804,4.098083,3.535171,1.552550
99997,1.049337,1.483073,3.130511,2.408602,0.982115
99998,3.932660,4.834658,3.405621,1.210054,7.815475


In [7]:
X = df.drop(['Y'], axis = 1)

In [8]:
X

,X1,X2,X3,X4
0,4.725219,1.735492,4.956655,1.984024
1,2.948350,3.917977,1.122311,2.076565
2,2.056012,4.279370,3.592577,2.878457
3,4.479831,4.364756,1.614712,3.498773
4,3.310144,1.695486,1.332648,2.947337
...,...,...,...,...
99995,3.596992,4.233839,1.935430,3.787543
99996,2.140851,2.861804,4.098083,3.535171
99997,1.049337,1.483073,3.130511,2.408602
99998,3.932660,4.834658,3.405621,1.210054


In [9]:
y = df['Y']
y

0        6.442611
1        3.268741
2        3.040191
3        7.153415
4        3.784738
           ...   
99995    6.270531
99996    1.552550
99997    0.982115
99998    7.815475
99999    3.695469
Name: Y, Length: 100000, dtype: float64

In [10]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((70000, 4), (30000, 4), (70000,), (30000,))

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
train_features = scaler.transform(X_train)
test_features = scaler.transform(X_test)


In [ ]:
test_features

array([[0.1827092 , 0.00573433, 0.71337059],
       [0.11174035, 0.17909944, 0.87801167],
       [0.65821597, 0.77453235, 0.96570729],
       ...,
       [0.64025318, 0.29137216, 0.4689263 ],
       [0.32775612, 0.1025758 , 0.15283047],
       [0.95199251, 0.5150051 , 0.44657199]])

In [11]:
est_gp = SymbolicRegressor(population_size=5000, generations=40, 
                           tournament_size=20, stopping_criteria=0.01, 
                           const_range=(- 1.0, 1.0), init_depth=(2, 6), 
                           init_method='half and half', 
                           #function_set=('add', 'sub', 'mul', 'div'), 
                           metric='mean absolute error', parsimony_coefficient=0.001, 
                           p_crossover=0.95, p_subtree_mutation=0.01, p_hoist_mutation=0.01, 
                           p_point_mutation=0.01, p_point_replace=0.05, max_samples=1.0, 
                           feature_names=X_train.columns, warm_start=False, low_memory=False, 
                           n_jobs=1, verbose=1, random_state=None)

In [12]:
est_gp.fit(X_train, y_train)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    30.34      1.64281e+07        7          1.46918              N/A     32.55m
   1     8.92          3.73927        7          1.41538              N/A     15.61m
   2     6.78          2.80803       13          1.39683              N/A     13.24m
   3     8.24          2.98726       15          1.36618              N/A     14.42m
   4     9.75          2.85805       13          1.34092              N/A     15.53m
   5    11.38           2.5892       17          1.26394              N/A     15.75m
   6    12.70          2.50623       31          1.30561              N/A     16.86m
   7    13.31            2.427       19           1.2878              N/A     17.91m
   8    14.15          2.52712       21          1.26328              N/A  

SymbolicRegressor(feature_names=Index(['X1', 'X2', 'X3', 'X4'], dtype='object'),
                  generations=40, p_crossover=0.95, population_size=5000,
                  stopping_criteria=0.01, verbose=1)

In [13]:
t0 = time.time()
y_gp1 = est_gp.predict(X_test)
print('Time to predict:', time.time() - t0, 'seconds')

Time to predict: 0.014410972595214844 seconds


In [14]:
score_gp1 = est_gp.score(X_test, y_test)
print('R2:', score_gp1)

R2: 0.6845885607897022


In [15]:
converter = {
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x*y,
    'div': lambda x, y : x/y,
    'sqrt': lambda x : x**0.5,
    'log': lambda x : log(x),
    'abs': lambda x : abs(x),
    'neg': lambda x : -x,
    'inv': lambda x : 1/x,
    'max': lambda x, y : max(x, y),
    'min': lambda x, y : min(x, y),
    'sin': lambda x : sin(x),
    'cos': lambda x : cos(x),
    'pow': lambda x, y : x**y,
}

In [ ]:
!pip install sympy==1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
next_e = sympify(str(est_gp._program), locals=converter)
next_e

0.515*X1 + 0.53045*X2 + 1.515*X3/X4 - 2.191705625 + 0.515*(X3/X4 - 0.998)*(X3 - X4**2 + (-0.7098*X1 - 0.3549*X2 - 1.0647*X3/X4 + 1.0625706 - 0.3549*(X3 - X4**2)*(X3/X4 - 0.998)/X3 - 0.456976*X4*(X2 - X4**2)/X3**2)/X3)/X3 - 0.34814*(X2 - X4**2)/X3**2

In [ ]:
str(est_gp._program)

'add(div(add(X1, div(div(-0.676, -0.258), div(X3, X1))), div(div(X3, X2), div(X2, mul(X3, div(-0.676, -0.258))))), X1)'

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

est_tree = DecisionTreeRegressor()
est_tree.fit(X_train, y_train)
est_rf = RandomForestRegressor(n_estimators=10)
est_rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10)

In [18]:
score_rf = est_rf.score(X_test, y_test)
score_rf

0.9942151276560968

In [19]:
score_dt = est_tree.score(X_test, y_test)
score_dt

0.9785313767114836

In [ ]:
import os
import random

os.chdir("/content/AI-Feynman/example_data")

def getY(m1,m2,x2,x1,y2,y1,z2,z1):
  y = 9.18*m1*m2/((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)
  return y

def getRow():
  [m1,m2,x2,x1,y2,y1,z2,z1]=[random.random() for x in range(8)]
  
  y=getY(m1,m2,x2,x1,y2,y1,z2,z1)
  return str(m1)+" "+str(m2)+" "+str(x2)+" "+str(x1)+" "+str(y2)+" "+str(y1)+" "+str(z2)+" "+str(z1)+"\n"

with open("duplicateVarsExample.txt", "w") as f:
  for _ in range(1000):
    f.write(getRow())
f.close()

# switch back to the code directory
os.chdir("/content/AI-Feynman/Code")
